In [2]:
%matplotlib notebook
from __future__ import division
from scipy.stats import binom as binomial
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from ipywidgets import StaticInteract, RangeWidget
import pandas as pd
from IPython.display import display
from sympy import init_printing, symbols, Eq
init_printing()

/home/saket/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [3]:
coin_toss = []
coin_toss.append('H T T T H H T H T H'.split())
coin_toss.append('H H H H T H H H H H'.split())
coin_toss.append('H T H H H H H T H H'.split())
coin_toss.append('H T H T T T H H T T'.split())
coin_toss.append('T H H H T H H H T H'.split())

In [4]:
df = pd.DataFrame(coin_toss)

In [5]:
df

,0,1,2,3,4,5,6,7,8,9
0,H,T,T,T,H,H,T,H,T,H
1,H,H,H,H,T,H,H,H,H,H
2,H,T,H,H,H,H,H,T,H,H
3,H,T,H,T,T,T,H,H,T,T
4,T,H,H,H,T,H,H,H,T,H


In [6]:
thetaA, thetaB = symbols('theta_A theta_B')
a,b = thetaA, thetaB # Hack to display
thetaA = 0.6
thetaB = 0.5
a , thetaA


In [7]:
b, thetaB

In [8]:
## Observed Case

observed = ['B', 'A', 'A', 'B', 'A']
index_A = [i for i,x in enumerate(observed) if x=='A']
index_B = [i for i,x in enumerate(observed) if x=='B']
total_tosses = df.size
A_tosses = df.iloc[index_A].unstack()
B_tosses = df.iloc[index_B].unstack()

A_heads = A_tosses.value_counts()['H']
B_heads = B_tosses.value_counts()['H']

theta_A = A_heads/A_tosses.size
theta_B = B_heads/B_tosses.size

In [9]:
(a, theta_A)

In [10]:
(b, theta_B)

In [11]:
thetaA = 0.6
thetaB = 0.5

def em(theta_old):
    row_prob = []
    ## Expectation
    for row in coin_toss:
        count_heads = row.count('H')
        p_a = binomial.pmf(count_heads, len(row), theta_old['A'])
        p_b = binomial.pmf(count_heads, len(row), theta_old['B'])
        p_t = p_a+p_b
        p_a = p_a/p_t
        p_b = p_b/p_t
        row_prob.append({'A': p_a, 'B': p_b, 'count_heads': count_heads, 'total_tosses': len(row)})
        
    ## Maximisation
    new_coin_toss = []
    for row in row_prob:
        total_tosses = row['total_tosses']
        total_heads = row['count_heads']
        A_heads = row['A']*total_heads
        A_tails = row['A']*(total_tosses-total_heads)
        B_heads = row['B']*total_heads
        B_tails = row['B']*(total_tosses-total_heads)
        new_coin_toss.append([[A_heads, A_tails], [B_heads, B_tails]])
    return new_coin_toss

theta = {'A': thetaA, 'B': thetaB}
em(theta)